In [194]:
import time

notebook_start_time = time.time()

# Set up environment

In [195]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/chinadupaya/art-recommendations.git
    %cd art-recommendations/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment


# 👩🏻‍🔬 Feature pipeline: Computing features

# Imports

In [196]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import warnings
from pprint import pprint

import polars as pl
import torch
from loguru import logger
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")

from recsys import hopsworks_integration
from recsys.config import settings
from recsys.features.artworks import (
    compute_features_artworks,
    generate_embeddings_for_dataframe,
)
from recsys.features.users import DatasetSampler, compute_features_users
from recsys.features.interaction import generate_interaction_data
from recsys.features.ranking import compute_ranking_dataset
from recsys.features.ranking3 import compute_ranking_dataset2
from recsys.features.transactions import compute_features_transactions
from recsys.hopsworks_integration import feature_store

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Constants

The most important one is the dataset size.

Choosing a different dataset size will impact the time it takes to run everything and the quality of the final models. We suggest using a small dataset size when running this the first time.

Suported user dataset sizes:

In [197]:
DatasetSampler.get_supported_sizes()

{<UserDatasetSize.LARGE: 'LARGE'>: 50000,
 <UserDatasetSize.MEDIUM: 'MEDIUM'>: 5000,
 <UserDatasetSize.SMALL: 'SMALL'>: 1000}

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [198]:
from dotenv import load_dotenv
import os
load_dotenv()
hopskey=os.environ["HOPSWORKS_API_KEY"]
project, fs = hopsworks_integration.get_feature_store(hopskey)

2024-12-25 22:18:59.879 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:15 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2024-12-25 22:18:59,880 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-25 22:18:59,883 INFO: Initializing external client
2024-12-25 22:18:59,884 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-25 22:19:01,302 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207464
2024-12-25 22:19:02,453 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-25 22:19:02,456 INFO: Initializing external client
2024-12-25 22:19:02,456 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-25 22:19:03,799 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207464


In [199]:
artworks_df =  pl.read_csv("../data/artworks_info.csv")
artworks_df.shape

(18538, 8)

The artworks DataFrame looks as follows

In [200]:
artworks_df.head(3)

id,title,category,thumbnail_link,artists_link,genes_link,similar_link,description
str,str,str,str,str,str,str,str
"""4d8b92eb4eb68a1b2c000968""","""Der Kuss (The Kiss)""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""https://api.artsy.net/api/arti…","""https://api.artsy.net/api/gene…","""https://api.artsy.net/api/artw…","""The image is rich in golds and…"
"""4d8b92ee4eb68a1b2c0009ab""","""The Third of May""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""https://api.artsy.net/api/arti…","""https://api.artsy.net/api/gene…","""https://api.artsy.net/api/artw…","""The image features a somber pa…"
"""4d8b93394eb68a1b2c0010fa""","""The Company of Frans Banning C…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""https://api.artsy.net/api/arti…","""https://api.artsy.net/api/gene…","""https://api.artsy.net/api/artw…","""The image features a rich pale…"


In [201]:
artworks_df.null_count()

id,title,category,thumbnail_link,artists_link,genes_link,similar_link,description
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


## Artworks Feature Engineering

In [202]:
artworks_df = compute_features_artworks(artworks_df)
artworks_df.shape


(18538, 5)

In [203]:
artworks_df.head()

artwork_id,title,category,thumbnail_link,description
str,str,str,str,str
"""4d8b92eb4eb68a1b2c000968""","""Der Kuss (The Kiss)""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image is rich in golds and…"
"""4d8b92ee4eb68a1b2c0009ab""","""The Third of May""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a somber pa…"
"""4d8b93394eb68a1b2c0010fa""","""The Company of Frans Banning C…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a rich pale…"
"""4d8b937c4eb68a1b2c001722""","""Mona Lisa""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image is characterized by …"
"""4d8b93b04eb68a1b2c001b9d""","""Luncheon on the Grass (Le Déje…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a palette o…"


### Create embeddings from the articles description

In [204]:
for i, desc in enumerate(artworks_df["description"].head(n=3)):
    logger.info(f"Item {i+1}:\n{desc}")

2024-12-25 22:19:09.392 | INFO     | __main__:<module>:2 - Item 1:
The image is rich in golds and soft pastels, creating a warm, harmonious atmosphere. The mood is intimate and tender, evoking feelings of love and closeness. The aesthetic features intricate patterns and floral motifs, reminiscent of the Art Nouveau style. Overall, it radiates a sense of tranquility and beauty.
2024-12-25 22:19:09.393 | INFO     | __main__:<module>:2 - Item 2:
The image features a somber palette dominated by dark tones and stark contrasts, highlighting the brutality of the scene. The mood conveys a sense of despair and urgency, evoking empathy for the victims. The dramatic lighting, particularly the lantern's glow, creates a tension between hope and tragedy. Overall, the aesthetic is visceral and haunting, capturing the raw emotion of human conflict.
2024-12-25 22:19:09.394 | INFO     | __main__:<module>:2 - Item 3:
The image features a rich palette of dark hues, contrasted by bright highlights, creatin

In [205]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
logger.info(
    f"Loading '{settings.FEATURES_EMBEDDING_MODEL_ID}' embedding model to {device=}"
)

# Load the embedding model from SentenceTransformer's model registry.
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2024-12-25 22:19:09.979 | INFO     | __main__:<module>:8 - Loading 'all-MiniLM-L6-v2' embedding model to device='cpu'


2024-12-25 22:19:09,980 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [206]:
embedding = generate_embeddings_for_dataframe(
    artworks_df.head(1), "description", model, batch_size=128
)['embeddings']
embedding

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00, 31.69it/s]


embeddings
list[f64]
"[-0.008282, 0.030209, … -0.003906]"


In [207]:
type(artworks_df)

polars.dataframe.frame.DataFrame

In [208]:
artworks_df = artworks_df.with_columns(
    pl.lit(embedding[0].to_list()).alias("embeddings")
)
artworks_df.head(3)

artwork_id,title,category,thumbnail_link,description,embeddings
str,str,str,str,str,list[f64]
"""4d8b92eb4eb68a1b2c000968""","""Der Kuss (The Kiss)""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image is rich in golds and…","[-0.008282, 0.030209, … -0.003906]"
"""4d8b92ee4eb68a1b2c0009ab""","""The Third of May""","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a somber pa…","[-0.008282, 0.030209, … -0.003906]"
"""4d8b93394eb68a1b2c0010fa""","""The Company of Frans Banning C…","""Painting""","""https://d32dm0rphc51dk.cloudfr…","""The image features a rich pale…","[-0.008282, 0.030209, … -0.003906]"


In [209]:
artworks_df = generate_embeddings_for_dataframe(
    artworks_df, "description", model, batch_size=128
)  # Reduce batch size if getting OOM errors.

Generating embeddings: 100%|██████████| 18538/18538 [03:12<00:00, 96.27it/s]


For each artwork description, we have a numerical vector which we can feed to a model, opposite to a string containing the description of an object.

In [210]:
artworks_df[["description", "embeddings"]].head(3)

description,embeddings
str,list[f64]
"""The image is rich in golds and…","[-0.008282, 0.030209, … -0.003906]"
"""The image features a somber pa…","[0.058479, 0.083046, … 0.008479]"
"""The image features a rich pale…","[0.012057, 0.037521, … 0.004547]"


## Looking at image links

In [211]:
artworks_df["thumbnail_link"][0]

'https://d32dm0rphc51dk.cloudfront.net/NOpIAwQa-3r51Cg9qXKbfA/medium.jpg'

In [212]:
from IPython.display import HTML, display

image_urls = artworks_df["thumbnail_link"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10px; max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))


# Users Data

In [213]:
users_df = pl.read_csv("../data/updated_user_details.csv")
users_df.shape


(54000, 5)

The users DataFrame looks as follows:

In [214]:
users_df.head(3)

user_id,literal,age,gender,preference
str,str,i64,str,str
"""69ff63b1-4803-49c9-8585-3b0f47…","""[Abbaye du Gard pres d' Abbevi…",42,"""Male""","""Impressionism"""
"""81cd5866-14fb-4968-80f2-accf84…","""[Italian Landscape, Sunset (So…",48,"""Female""","""Impressionism"""
"""83e00911-6b5a-44d3-9edb-3c02a2…","""[The Church of Souain, The Sei…",56,"""Female""","""Impressionism"""


Check for nulls

In [215]:
users_df.null_count()

user_id,literal,age,gender,preference
u32,u32,u32,u32,u32
0,0,0,0,0


# Users feature engineering

In [216]:
users_df = compute_features_users(users_df, drop_null_age=True)
users_df.shape

(54000, 6)

# Transactions Data
These are the artworks a user likes

In [217]:
transactions_df = pl.read_csv("../data/transaction-data.csv")
transactions_df.shape

(306100, 4)

Transactions DataFrame looks as follows

In [218]:
transactions_df.head(3)

transaction_id,user_id,artwork_id,thumbnail_link
str,str,str,str
"""f9f34e02-f51d-42ff-ada2-928728…","""636d1fa7-a3c2-4fe6-b278-265d29…","""516cbb4a0f8b7853440003fe""","""https://d32dm0rphc51dk.cloudfr…"
"""7f8ba386-bfe0-4c0a-b71e-c45c9b…","""8cea0e10-c960-44e9-b404-bc3047…","""516cbfd89ad2d38886000142""","""https://d32dm0rphc51dk.cloudfr…"
"""a2f1abf6-b58b-4f4a-a0ba-ee2fba…","""5ab9bc35-c1e5-4d99-bddd-499653…","""515b45c0223afa29bd000948""","""https://d32dm0rphc51dk.cloudfr…"


In [219]:
from datetime import datetime
t_dat = datetime.today()
# t_dat = datetime.today().strftime('%Y-%m-%d')
transactions_df = (
    transactions_df.with_columns(t_dat=pl.lit(t_dat))
)

transactions_df.head(3)

transaction_id,user_id,artwork_id,thumbnail_link,t_dat
str,str,str,str,datetime[μs]
"""f9f34e02-f51d-42ff-ada2-928728…","""636d1fa7-a3c2-4fe6-b278-265d29…","""516cbb4a0f8b7853440003fe""","""https://d32dm0rphc51dk.cloudfr…",2024-12-25 22:22:27.650584
"""7f8ba386-bfe0-4c0a-b71e-c45c9b…","""8cea0e10-c960-44e9-b404-bc3047…","""516cbfd89ad2d38886000142""","""https://d32dm0rphc51dk.cloudfr…",2024-12-25 22:22:27.650584
"""a2f1abf6-b58b-4f4a-a0ba-ee2fba…","""5ab9bc35-c1e5-4d99-bddd-499653…","""515b45c0223afa29bd000948""","""https://d32dm0rphc51dk.cloudfr…",2024-12-25 22:22:27.650584


# Transactions Feature Engineering

In [220]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(306100, 9)

In [221]:
transactions_df.head(3)

transaction_id,user_id,artwork_id,thumbnail_link,t_dat,year,month,day,day_of_week
str,str,str,str,i64,i32,i8,i8,i8
"""f9f34e02-f51d-42ff-ada2-928728…","""636d1fa7-a3c2-4fe6-b278-265d29…","""516cbb4a0f8b7853440003fe""","""https://d32dm0rphc51dk.cloudfr…",1735165347,2024,12,25,3
"""7f8ba386-bfe0-4c0a-b71e-c45c9b…","""8cea0e10-c960-44e9-b404-bc3047…","""516cbfd89ad2d38886000142""","""https://d32dm0rphc51dk.cloudfr…",1735165347,2024,12,25,3
"""a2f1abf6-b58b-4f4a-a0ba-ee2fba…","""5ab9bc35-c1e5-4d99-bddd-499653…","""515b45c0223afa29bd000948""","""https://d32dm0rphc51dk.cloudfr…",1735165347,2024,12,25,3


We don't want to work with ~300k transactions in these series, as everything will take too much time to run. Thus, we create a subset of the original dataset by randomly sampling from the users' datasets and taking only their transactions.

In [224]:
len(transactions_df)

306100

In [178]:
sampler = DatasetSampler(size=settings.USER_DATA_SIZE)
dataset_subset = sampler.sample(
    users_df=users_df, transations_df=transactions_df, artworks_df=artworks_df
)
users_df = dataset_subset["users"]
transactions_df = dataset_subset["transactions"]

2024-12-25 03:31:54.481 | INFO     | recsys.features.users:sample:29 - Sampling 900 users.
2024-12-25 03:31:54.484 | INFO     | recsys.features.users:sample:32 - Number of transactions for all the transactions: 306100
2024-12-25 03:31:54.499 | INFO     | recsys.features.users:sample:38 - Number of users for the 900 sampled users: 54000
2024-12-25 03:31:54.501 | INFO     | recsys.features.users:sample:44 - Number of artworks for the 900 sampled users: 900


In [225]:
transactions_df.shape

(306100, 9)

Some of the remaining customers

In [226]:
for user_id in transactions_df["user_id"].unique().head(10):
    logger.info(f"Logging user ID: {user_id}")

2024-12-25 22:23:40.287 | INFO     | __main__:<module>:2 - Logging user ID: 71c46bed-cffd-4375-aebe-5e6926a3b0ee
2024-12-25 22:23:40.288 | INFO     | __main__:<module>:2 - Logging user ID: 7860176e-050d-458d-901d-876aaa704986
2024-12-25 22:23:40.289 | INFO     | __main__:<module>:2 - Logging user ID: a04ff180-5a29-4384-84f2-ebe4b2d94c17
2024-12-25 22:23:40.290 | INFO     | __main__:<module>:2 - Logging user ID: 97edeb2e-2be2-4e67-a500-c08f4a77b2f6
2024-12-25 22:23:40.291 | INFO     | __main__:<module>:2 - Logging user ID: 0e794d57-c0df-4705-96a3-d8c8320f83fa
2024-12-25 22:23:40.292 | INFO     | __main__:<module>:2 - Logging user ID: 922bc7e4-6da8-45ea-95b1-d11a87b45bf0
2024-12-25 22:23:40.293 | INFO     | __main__:<module>:2 - Logging user ID: a2d9a51e-8a02-4b80-8151-585630d1e196
2024-12-25 22:23:40.293 | INFO     | __main__:<module>:2 - Logging user ID: c7ee937b-6d27-451e-b46a-1e401c8442de
2024-12-25 22:23:40.294 | INFO     | __main__:<module>:2 - Logging user ID: 31e84933-ac48-4a58-8

# 🤳🏻 Interaction data

To train our models, we need more than just the transactions DataFrame. We need positive samples that signal whether a user clicked or liked an item, but we also need negative samples that signal no interactions between a user and an artwork.

In [227]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing user chunks: 100%|██████████| 54/54 [05:32<00:00,  6.15s/it]


(5058388, 5)

The interaction features look as follows:

In [228]:
interaction_df.head()

t_dat,user_id,artwork_id,interaction_score,prev_artwork_id
i64,str,str,i64,str
1360765347,"""00062411-7dfa-4e40-a892-db020e…","""5328795b0bb6d6bf0b00032a""",0,"""START"""
1385965347,"""00062411-7dfa-4e40-a892-db020e…","""5328795b0bb6d6bf0b00032a""",0,"""5328795b0bb6d6bf0b00032a"""
1414765347,"""00062411-7dfa-4e40-a892-db020e…","""515d61935eeb1c904c00464b""",0,"""5328795b0bb6d6bf0b00032a"""
1421965347,"""00062411-7dfa-4e40-a892-db020e…","""515b4d5a223afaab8f001608""",0,"""515d61935eeb1c904c00464b"""
1425565347,"""00062411-7dfa-4e40-a892-db020e…","""515d2a6e7696593fde0034e3""",0,"""515b4d5a223afaab8f001608"""


Let's take a look at the interaction score distribution

In [229]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,4005105
1,747183
2,306100


Here is what each score means:
- `0` : No interaction between a user and an artwork
- `1` : A user clicked an artwork
- `2` : A cuser liked an item

# <span style="color:#ff5f27">🪄 Create Hopsworks Feature Groups </span>

A [feature group](https://docs.hopsworks.ai/feature-store-api/latest/generated/feature_group/) can be seen as a collection of conceptually related features.

To create a feature group you need to give it a name and specify a primary key. It is also best practice to provide a description of the contents of the feature group.

#### Users

We set `online_enabled=True` to enable low-latency access to the data from the inference pipeline for real-time predictions. 

A full list of arguments can be found in the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/feature_store_api/#create_feature_group).

In [230]:
logger.info("Uploading 'users' Feature Group to Hopsworks.")
users_fg = feature_store.create_users_feature_group(
    fs, df=users_df, online_enabled=True
)

logger.info("✅ Uploaded 'users' Feature Group to Hopsworks!")

2024-12-25 22:29:25.777 | INFO     | __main__:<module>:1 - Uploading 'users' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 54000/54000 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207464/jobs/named/users_1_offline_fg_materialization/executions
2024-12-25 22:29:48,450 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-25 22:29:51,621 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-25 22:31:20,554 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-25 22:31:20,783 INFO: Waiting for log aggregation to finish.
2024-12-25 22:31:29,471 INFO: Execution finished successfully.


2024-12-25 22:31:32.458 | INFO     | __main__:<module>:6 - ✅ Uploaded 'users' Feature Group to Hopsworks!


#### Artworks

Let's do the same thing for the rest of the data frames

In [231]:
logger.info("Uploading 'artworks' Feature Group to Hopsworks.")
artworks_fg = feature_store.create_artworks_feature_group(
    fs,
    df=artworks_df,
    artworks_description_embedding_dim=model.get_sentence_embedding_dimension(),
    online_enabled=True,
)
logger.info("✅ Uploaded 'artworks' Feature Group to Hopsworks!")


2024-12-25 22:31:33.081 | INFO     | __main__:<module>:1 - Uploading 'artworks' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 18538/18538 | Elapsed Time: 00:30 | Remaining Time: 00:00


Launching job: artworks_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207464/jobs/named/artworks_1_offline_fg_materialization/executions
2024-12-25 22:32:18,225 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-25 22:32:21,399 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-25 22:34:03,089 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-25 22:34:03,288 INFO: Waiting for log aggregation to finish.
2024-12-25 22:34:11,813 INFO: Execution finished successfully.


2024-12-25 22:34:11.815 | INFO     | __main__:<module>:8 - ✅ Uploaded 'artworks' Feature Group to Hopsworks!


#### Transactions

In [232]:
logger.info("Uploading 'transactions' Feature Group to Hopsworks.")
trans_fg = feature_store.create_transactions_feature_group(
    fs=fs, df=transactions_df, online_enabled=True
)
logger.info("✅ Uploaded 'transactions' Feature Group to Hopsworks!")

2024-12-25 22:34:12.407 | INFO     | __main__:<module>:1 - Uploading 'transactions' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 306100/306100 | Elapsed Time: 00:38 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207464/jobs/named/transactions_1_offline_fg_materialization/executions
2024-12-25 22:35:05,960 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-25 22:35:09,134 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-25 22:36:47,651 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2024-12-25 22:36:54,000 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2024-12-25 22:36:54,409 INFO: Waiting for log aggregation to finish.
2024-12-25 22:36:54,410 INFO: Execution finished successfully.


2024-12-25 22:36:58.917 | INFO     | __main__:<module>:5 - ✅ Uploaded 'transactions' Feature Group to Hopsworks!


#### Interactions

In [233]:
logger.info("Uploading 'interactions' Feature Group to Hopsworks.")
interactions_fg = feature_store.create_interactions_feature_group(
    fs=fs, df=interaction_df, online_enabled=True
)
logger.info("✅ Uploaded 'interactions' Feature Group to Hopsworks!!")

2024-12-25 22:36:59.493 | INFO     | __main__:<module>:1 - Uploading 'interactions' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 5058388/5058388 | Elapsed Time: 07:35 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207464/jobs/named/interactions_1_offline_fg_materialization/executions
2024-12-25 22:44:51,079 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-25 22:44:54,255 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-25 22:55:13,993 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-25 22:55:14,198 INFO: Waiting for log aggregation to finish.
2024-12-25 22:55:22,884 INFO: Execution finished successfully.


2024-12-25 22:55:25.975 | INFO     | __main__:<module>:5 - ✅ Uploaded 'interactions' Feature Group to Hopsworks!!


In [234]:
# import hopsworks 
# project = hopsworks.login(api_key_value="H")
# fs = project.get_feature_store(name='id2223artsy_featurestore')
artworks_fg = fs.get_feature_group('artworks', version=1)

# Compute ranking dataset

The last step is to compute the ranking dataset used to train the scoring/ranking model from the feature groups we've just created:


In [235]:
# ranking_df = compute_ranking_dataset(
#     trans_fg,
#     artworks_fg,
#     users_fg,
# )
# ranking_df.shape

The ranking dataset looks as follows:

In [236]:
ranking_df = compute_ranking_dataset2(
    transactions_df,
    artworks_df,
    users_df,
)
ranking_df.shape

(3367100, 7)

In [237]:
ranking_df.head(3)

user_id,age,artwork_id,label,title,description,category
str,f64,str,i32,str,str,str
"""636d1fa7-a3c2-4fe6-b278-265d29…",49.0,"""516cbb4a0f8b7853440003fe""",1,"""Christ on the Cross""","""The image features a muted pal…","""Print"""
"""8cea0e10-c960-44e9-b404-bc3047…",28.0,"""516cbfd89ad2d38886000142""",1,"""The Rest on the Flight into Eg…","""The image is characterized by …","""Print"""
"""5ab9bc35-c1e5-4d99-bddd-499653…",26.0,"""515b45c0223afa29bd000948""",1,"""Saint Paul""","""The image features a muted pal…","""Print"""


In [238]:
ranking_df.get_column("label").value_counts()

label,count
i32,u32
0,3061000
1,306100


In [239]:
logger.info("Uploading 'ranking' Feature Group to Hopsworks.")
rank_fg = feature_store.create_ranking_feature_group(
    fs,
    df=ranking_df,
    parents=[artworks_fg, users_fg, trans_fg],
    online_enabled=False
)
logger.info("✅ Uploaded 'ranking' Feature Group to Hopsworks!!")

2024-12-25 22:55:30.458 | INFO     | __main__:<module>:1 - Uploading 'ranking' Feature Group to Hopsworks.
Uploading Dataframe: 100.00% |██████████| Rows 3367100/3367100 | Elapsed Time: 13:16 | Remaining Time: 00:00


Launching job: ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207464/jobs/named/ranking_1_offline_fg_materialization/executions
2024-12-25 23:09:04,470 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-25 23:09:07,640 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-25 23:21:53,768 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2024-12-25 23:21:56,936 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-25 23:21:57,141 INFO: Waiting for log aggregation to finish.
2024-12-25 23:22:05,828 INFO: Execution finished successfully.


2024-12-25 23:22:10.046 | INFO     | __main__:<module>:8 - ✅ Uploaded 'ranking' Feature Group to Hopsworks!!
